In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
from string import punctuation
from random import shuffle

import gensim
from gensim.models.word2vec import Word2Vec
LabeledSentence = gensim.models.doc2vec.LabeledSentence

from keras.utils import np_utils

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, Activation, LSTM, Embedding
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
def ingest():
    data = pd.read_csv("mydata.csv",error_bad_lines=False, encoding = "ISO-8859-1",header=None)
    data.columns = ["SentimentText","Sentiment"]
    data['Sentiment'] = data['Sentiment'].map(int)
    return data

data = ingest()
pos = data[data.Sentiment == 1]
neg = data[data.Sentiment == 0]
print("Pos examples:",pos.shape,"Neg examples:", neg.shape)

Pos examples: (27374, 2) Neg examples: (8344, 2)


In [4]:
def tokenize(tweet):
    try:
        tweet = str(tweet).lower()
        tokens = tokenizer.tokenize(tweet)
        tokens = [ token for token in tokens if not ( token.startswith("@") or token.startswith("http") or token.startswith("#")) ]
        return tokens
    except:
        return None

In [5]:
def postprocess(data):
    data['tokens'] = data['SentimentText'].progress_map(tokenize)
    return data

data = postprocess(data)

progress-bar: 100%|██████████| 35718/35718 [00:01<00:00, 18091.23it/s]


In [6]:
n = data.shape[0]

x_train, x_test, y_train, y_test = train_test_split(np.array(data.head(n).tokens),
                                                    np.array(data.head(n).Sentiment), test_size=0.2)


In [11]:
def labelizeTweets(tweets, label_type):
    labelized = []
    for i,v in tqdm(enumerate(tweets)):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

x_train = labelizeTweets(x_train, 'TRAIN')
x_test = labelizeTweets(x_test, 'TEST')



0it [00:00, ?it/s]

28574it [00:00, 443543.23it/s]

0it [00:00, ?it/s]

7144it [00:00, 469414.06it/s]

In [7]:
tweet_w2v = gensim.models.KeyedVectors.load_word2vec_format("dataset/google.bin.gz", binary=True)

In [8]:
def wordvector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += tweet_w2v[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [12]:
from sklearn.preprocessing import scale
n_dim = 300
train_vecs_w2v = np.concatenate([wordvector(z, n_dim) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([wordvector(z, n_dim) for z in tqdm(map(lambda x: x.words, x_test))])
test_vecs_w2v = scale(test_vecs_w2v)



0it [00:00, ?it/s]

1595it [00:00, 15943.06it/s]

3240it [00:00, 16090.46it/s]

4896it [00:00, 16228.04it/s]

6570it [00:00, 16360.04it/s]

8249it [00:00, 16486.61it/s]

9912it [00:00, 16527.99it/s]

11531it [00:00, 16424.71it/s]

13179it [00:00, 16438.56it/s]

14827it [00:00, 16449.59it/s]

16469it [00:01, 16439.76it/s]

18090it [00:01, 16369.54it/s]

19737it [00:01, 16397.77it/s]

21364it [00:01, 16357.36it/s]

22987it [00:01, 16317.37it/s]

24627it [00:01, 16341.82it/s]

26271it [00:01, 16370.91it/s]

27969it [00:01, 16548.76it/s]

28574it [00:01, 16446.98it/s]

0it [00:00, ?it/s]

1647it [00:00, 16467.54it/s]

3223it [00:00, 16245.43it/s]

4852it [00:00, 16256.33it/s]

6459it [00:00, 16197.91it/s]

7144it [00:00, 16064.67it/s]

In [13]:
train_y = np_utils.to_categorical(y_train)
test_y = np_utils.to_categorical(y_test)

In [20]:
model = Sequential()
model.add(Dense(512, activation='relu', input_dim=300, kernel_initializer='normal'))
model.add(Dense(512, activation='relu', kernel_initializer='normal'))
model.add(Dense(2, activation='softmax', kernel_initializer='normal'))
model.compile(optimizer=Adam(lr=0.0003),
              loss='mean_squared_error',
              metrics=['accuracy'])
model.fit(train_vecs_w2v, train_y, validation_data=(test_vecs_w2v, test_y), epochs=30, batch_size=100, verbose=1)

Train on 28574 samples, validate on 7144 samples
Epoch 1/30
28574/28574 [==============================] - 2s - loss: 0.0248 - acc: 0.9706 - val_loss: 0.0044 - val_acc: 0.9961
Epoch 2/30
28574/28574 [==============================] - 1s - loss: 0.0031 - acc: 0.9970 - val_loss: 0.0033 - val_acc: 0.9968
Epoch 3/30
28574/28574 [==============================] - 1s - loss: 0.0020 - acc: 0.9980 - val_loss: 0.0024 - val_acc: 0.9978
Epoch 4/30
28574/28574 [==============================] - 1s - loss: 0.0016 - acc: 0.9984 - val_loss: 0.0023 - val_acc: 0.9978
Epoch 5/30
28574/28574 [==============================] - 1s - loss: 0.0016 - acc: 0.9984 - val_loss: 0.0023 - val_acc: 0.9978
Epoch 6/30
28574/28574 [==============================] - 1s - loss: 0.0016 - acc: 0.9984 - val_loss: 0.0022 - val_acc: 0.9978
Epoch 7/30
28574/28574 [==============================] - 1s - loss: 0.0016 - acc: 0.9984 - val_loss: 0.0022 - val_acc: 0.9978
Epoch 8/30
28574/28574 [==============================] - 1s -

In [26]:
m = 108
print(data.SentimentText[m], data.Sentiment[m])
scentence = data.tokens[m]
scentence = tokenize("Kovind will make an exceptional president: PM Narendra Modi")
vec = wordvector(scentence, 300)
print(model.predict(vec))

To make the #InternationalYogaDay programme in #Lucknow a success, various NGOs and #Yoga gurus have camped there. https://t.co/TNuAucVF2Q 1
[[ 0.35325179  0.64674824]]


In [27]:
model.save("model1.h5")